In [ ]:
london = 3292.84
er = 7.3
print(london/31.1035*er)

In [1]:
import json 
import torch 
import numpy as np 

In [ ]:
def recursive_to_tensor(obj):
    if isinstance(obj, dict):
        return {k: recursive_to_tensor(v) for k, v in obj.items()} 
    elif isinstance(obj, list):
        return torch.tensor(obj)   # 直接转为Tensor（避免numpy中间步骤）
    return obj

In [3]:

name = "./results/cifar_10/org_2.json"

def serializable_to_tensor(obj):
    # 检测到列表时尝试转为numpy数组，再转Tensor 
    if isinstance(obj, list):
        return torch.from_numpy(np.array(obj)) 
    return obj 
 
with open(name, 'r', encoding='utf-8') as f:
    loaded_data = recursive_to_tensor(json.load(f)) 

In [4]:
print(loaded_data)

{'train': {'loss': tensor([1.8111, 1.1371]), 'acc': tensor([0.4198, 0.5967])}, 'valid': {'loss': tensor([1.3131, 1.0338]), 'acc': tensor([0.5271, 0.6353])}, 'bast_acc': 0.6353}
